<a href="https://colab.research.google.com/github/woo2201/sampleCases/blob/reservior/SpaceTitanic_v3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/dataset/SpaceTitanic/

/content/drive/MyDrive/dataset/SpaceTitanic


In [3]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
path = "/content/drive/MyDrive/dataset/SpaceTitanic/"
filenames = glob.glob(path + "*.csv")
data = [var.split('.')[0].split('/')[-1] for var in filenames]

# dynamically allocate df names.
df_list = []
for name in zip(filenames, data):
    loc = list(name)[0]
    val = list(name)[1]
    if not val.startswith('HomeCredit'):
        df_list.append(val)
        globals()[val] = pd.read_csv(loc) # encoding = 'ISO-8859-1'

In [5]:
(train.shape[0] == train.PassengerId.nunique(), test.shape[0] == test.PassengerId.nunique())

(True, True)

In [12]:
def create_adversarial_validation_set(df_train, df_test, drop_cols, target):
    
    df_train['is_train'] = 1
    df_test['is_train'] = 0

    df_target = df_train[target]

    cols_to_drop = drop_cols
    cols = [i for i in train.columns if i not in cols_to_drop]
    df_combined = df_train[cols].append(df_test[cols])
    
    return df_combined, df_target

In [14]:
combined, target = create_adversarial_validation_set(train, test, ['PassengerId', 'Name', 'Transported'], 'Transported')

In [ ]:
def feature_engineering(df, dummyVariables):

    if isinstance(df, pd.DataFrame) and isinstance(dummyVariables, list):

        df = (
                df
                .assign(Psng_grp_1 = lambda x : x.PassengerId.str.split("_").apply(lambda x: x[0]))
                .assign(Psng_grp_2 = lambda x : x.PassengerId.str.split("_").apply(lambda x: x[1]))
        )

        CameAlone = [i for i, j in zip(df.Psng_grp_1.value_counts().index, df.Psng_grp_1.value_counts().values) if j == 1]

        df = (  df
                .assign(totalSpent = lambda x : x.RoomService	+ x.FoodCourt + x.ShoppingMall + x.Spa	+ x.VRDeck)
                .assign(Accompanied = lambda x : x.Psng_grp_1.apply(lambda x: 'Alone' if x in CameAlone else 'Accompanied'))
                .assign(Cabin_Type = lambda x : x.Cabin.str.split('/').apply(lambda x: x[2] if isinstance(x, list) else np.NaN))
                .assign(underAge = lambda x: x.Age.apply(lambda x: 1 if x <= 19 else 0))
        )
    
        df = pd.get_dummies(df, columns = dummyVariables)

    return df

In [ ]:
train_df = feature_engineering(train, dummyVariables = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination'])
test_df = feature_engineering(test, dummyVariables = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination'])

In [ ]:
train_keys = train_df[['PassengerId', 'Psng_grp_1', 'Psng_grp_2']]
test_keys = test_df[['PassengerId', 'Psng_grp_1', 'Psng_grp_2']]

target = train_df.Transported

In [ ]:
cols_to_drop = ['Cabin', 'Name', 'PassengerId', 'Psng_grp_1', 'Psng_grp_2']
for df in [train_df, test_df]:
    df.drop(cols_to_drop, axis = 1, inplace = True)
    df['Cabin_Type'] = df.Cabin_Type.apply(lambda x: 1 if x == 'P' else 0)
    df['Accompanied'] = df.Accompanied.apply(lambda x: 1 if x == 'Accompanied' else 0)

In [ ]:
train_df.drop(['Transported'], axis = 1, inplace = True)

XGBOOST only takes numerical values and is not able to measure the distance between any order-based values. 

In [ ]:
xgb_model = XGBRegressor(objective = "reg:squarederror",
                         learning_Rate = 0.05,
                         max_depth = 3,
                         n_estimators = 200)
xgb_model.fit(train_df, target, verbose = False)

XGBRegressor(learning_Rate=0.05, n_estimators=200, objective='reg:squarederror')

In [ ]:
# make predictions
predictions =  xgb_model.predict(test_df)

In [ ]:
predictions_binary = [True if x >= 0.5 else False for x in predictions]

In [ ]:
from collections import Counter
Counter(predictions_binary)

Counter({False: 1990, True: 2287})

In [ ]:
sample_submission.Transported = predictions_binary

In [ ]:
sample_submission.to_csv("rslt_1.csv", index = False)

# Logistic

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(max_iter = 1000)

In [ ]:
train_df.isna().mean()

Age                          0.020591
RoomService                  0.020821
FoodCourt                    0.021051
ShoppingMall                 0.023927
Spa                          0.021051
VRDeck                       0.021627
totalSpent                   0.104452
Accompanied                  0.000000
Cabin_Type                   0.000000
underAge                     0.000000
HomePlanet_Earth             0.000000
HomePlanet_Europa            0.000000
HomePlanet_Mars              0.000000
CryoSleep_False              0.000000
CryoSleep_True               0.000000
VIP_False                    0.000000
VIP_True                     0.000000
Destination_55 Cancri e      0.000000
Destination_PSO J318.5-22    0.000000
Destination_TRAPPIST-1e      0.000000
dtype: float64

In [ ]:
train_df = train_df.fillna(train_df.median())
test_df = test_df.fillna(test_df.median())

In [ ]:
logisticRegr.fit(train_df, target)

LogisticRegression(max_iter=1000)

In [ ]:
predictions = logisticRegr.predict(test_df)

In [ ]:
predictions

array([ True, False,  True, ...,  True,  True,  True])

In [ ]:
from collections import Counter
Counter(predictions)

Counter({False: 1988, True: 2289})

In [ ]:
sample_submission['Transported'] = predictions

In [ ]:
sample_submission.to_csv("./rslt/rslt_2.csv", index = False)